<a href="https://colab.research.google.com/github/ashesh-12/Advanced-Chatbot/blob/main/Chapter1_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!sudo apt-get remove python3-blinker

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-packagekitglib-1.0 libappstream4
  libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0
  libstemmer0d libxmlb2 libyaml-0-2 lsb-release packagekit pkexec policykit-1
  polkitd python-apt-common python3-apt python3-cffi-backend
  python3-cryptography python3-dbus python3-distro python3-gi python3-httplib2
  python3-jeepney python3-jwt python3-keyring python3-lazr.uri
  python3-pkg-resources python3-pyparsing python3-secretstorage python3-six
  python3-wadllib
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  python3-blinker python3-launchpadlib python3-lazr.restfulclient
  python3-oauthlib python3-software-properties software-properties-common
0 upgraded, 0 newly installed, 6 to remove and 0 not upgraded.
After this operation, 3,022 kB disk s

In [11]:
!pip install flask blinker

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)


In [12]:
!pip install transformers==4.41.2 # Replace with an appropriate version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 102.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [13]:
%%writefile flask_chatbot_app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Load the Hugging Face model

chatbot_model = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
conversation_history = []

@app.route('/chatbot', methods=['POST'])
def chatbot():
    user_input = request.json.get('message')
    if not user_input:
        return jsonify({'response': 'Please provide a message.'}), 400
    response = generate_response(user_input)
    return jsonify({'response': response})

def generate_response(user_input):
    # Generate a response using the Hugging Face model
    conversation_history.append(
        {"role": "user", "content": user_input},
    )
    result = chatbot_model(conversation_history, num_return_sequences=1, max_new_tokens=250)
    conversation_history.append(
        {"role": "assistant", "content": result[0]['generated_text'][-1]['content']},
    )
    return result[0]['generated_text'][-1]['content']

if __name__ == '__main__':
    app.run(debug=True, port=5000, host='0.0.0.0')


Overwriting flask_chatbot_app.py


In [14]:

import subprocess

# Stop any running Flask app
subprocess.run(['pkill', '-f', 'flask_chatbot_app.py'])

CompletedProcess(args=['pkill', '-f', 'flask_chatbot_app.py'], returncode=1)

In [15]:
!nohup python flask_chatbot_app.py &

nohup: appending output to 'nohup.out'


In [22]:
!sudo lsof -i -P -n | grep LISTEN

node         7 root   21u  IPv6  47262      0t0  TCP *:8080 (LISTEN)
kernel_ma   18 root    3u  IPv4  63598      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil  201 root    3u  IPv4  21049      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n  222 root    7u  IPv4  21057      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3    356 root   28u  IPv4  28990      0t0  TCP 127.0.0.1:41947 (LISTEN)
python3    450 root    3u  IPv4  58536      0t0  TCP 127.0.0.1:45001 (LISTEN)
python3    450 root    5u  IPv4  58537      0t0  TCP 127.0.0.1:53403 (LISTEN)
python3   1482 root    7u  IPv4  39559      0t0  TCP *:5000 (LISTEN)
python3   1961 root    6u  IPv4  39559      0t0  TCP *:5000 (LISTEN)
python3   1961 root    7u  IPv4  39559      0t0  TCP *:5000 (LISTEN)


In [23]:

import requests

# Define the URL of the Flask app
url = 'http://127.0.0.1:5000/chatbot'

# Send a request to the Flask app
response = requests.post(url, json={'message': 'Hello, how are you?'})
print(response.json())


{'response': " I'm doing well. How about you? How can I help you today?"}


In [ ]:
response = requests.post(url, json={'message': 'I was wondering how can I go to Eiffel Tower from the airport using the train and subway?'})
print(response.json())

{'response': " To travel from Charles de Gaulle Airport (CDG) to the Eiffel Tower using the train and subway, follow these steps:\n\n\n1. **Airport to Gare de l'Est Train Station:**\n\n   - Take the RER B train from Terminal 2 or Terminal 1 to Gare de l'Est. The train ride takes approximately 30 minutes.\n\n   - The train station is located inside Terminal 2, so you'll need to take an airport shuttle bus from Terminal 1 to Terminal 2.\n\n\n2. **Gare de l'Est to Eiffel Tower:**\n\n   - Once you arrive at Gare de l'Est, you'll need to transfer to the Metro.\n\n   - Take Metro Line 1 towards Château de Vincennes and get off at Champ de Mars-Tour Eiffel.\n\n   - The ride on the Metro takes about 15 minutes.\n\n   - The Eiffel Tower is located at Champ de Mars, which is a short walk from the station.\n\n\nPlease note that you'll need to purchase a"}


In [ ]:
response = requests.post(url, json={'message': 'And how do I go to Arche du Triomphe from there?'})
print(response.json())

{'response': " To travel from the Eiffel Tower to the Arc de Triomphe, follow these steps:\n\n\n1. **Eiffel Tower to Arc de Triomphe:**\n\n   - From the Eiffel Tower, you have a couple of options to reach the Arc de Triomphe.\n\n   - **Option 1: Walking:** The Arc de Triomphe is a 20-minute walk from the Eiffel Tower. Head west on Avenue de la Bourdonnais, then turn left onto Avenue de la Grande Armée, and continue straight until you reach the Arc de Triomphe.\n\n   - **Option 2: Metro:**\n\n     - Take Metro Line 1 from Champ de Mars-Tour Eiffel to Charles de Gaulle - Étoile.\n\n     - Exit at the Charles de Gaulle - Étoile station.\n\n     - From there, walk west along Avenue de la Grande Armée for about 10 minutes to reach the Arc de Triomphe.\n\n\nPlease note that the Metro option may be more convenient if you prefer not to walk, especially if you're carrying lug"}
